In [1]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

In [24]:
engine = create_engine("sqlite:///Data/team_database.db")

In [25]:
training_data = pd.read_sql("WIN_NUM_ROSTER_CHANGE_DATA", engine)

In [28]:
print(training_data.columns)

Index(['SEASON', 'TEAM_ID', 'TEAM_ABBREV', 'CHANGE_NUM_WINS', 'CHANGE_ROSTERS',
       'CHANGE_NET_RTG'],
      dtype='object')


In [49]:
ranges = []
prob_increase = []
average_increase = []
average_change = []
net_rating_range = []
for lower_bound_net_rating in range(0, 10, 5):
    for i in range(0, 100, 10):
        lower_bound = float(i) / 100
        upper_bound = (float(i) + 10) / 100

        teams = training_data[training_data["CHANGE_ROSTERS"] >= lower_bound]
        teams = teams[teams["CHANGE_ROSTERS"] < upper_bound]

        teams = teams[teams["CHANGE_NET_RTG"] >= lower_bound_net_rating]
        teams = teams[teams["CHANGE_NET_RTG"] < (lower_bound_net_rating + 5)]

        if len(teams) > 0:
            ranges.append("[" + str(lower_bound) + ", " + str(upper_bound) + ")")
            prob_increase.append(len(teams[teams["CHANGE_NUM_WINS"] > 0]) / len(teams))
            average_increase.append(np.mean(teams[teams["CHANGE_NUM_WINS"] > 0]["CHANGE_NUM_WINS"]))
            average_change.append(np.mean(teams["CHANGE_NUM_WINS"]))
            net_rating_range.append("[" + str(lower_bound_net_rating) + ", " + str(lower_bound_net_rating + 5) + "]")
            

In [50]:
df = pd.DataFrame({"ROSTER_CHANGE_INTERVAL": ranges, "PROB_INCREASE_WINS": prob_increase, "AVG_INCREASE": average_increase, "AVG_CHANGE": average_change, "NET_RTG_CHANGE": net_rating_range})

In [51]:
print(df)

   ROSTER_CHANGE_INTERVAL  PROB_INCREASE_WINS  AVG_INCREASE  AVG_CHANGE  \
0              [0.0, 0.1)            1.000000      3.000000    3.000000   
1              [0.1, 0.2)            0.700000      6.000000    2.500000   
2              [0.2, 0.3)            0.941176      6.937500    6.352941   
3              [0.3, 0.4)            0.871795      6.764706    5.410256   
4              [0.4, 0.5)            0.772727      7.921569    5.454545   
5              [0.5, 0.6)            0.800000      6.791667    4.850000   
6              [0.6, 0.7)            0.692308      7.259259    3.974359   
7              [0.7, 0.8)            0.777778      6.571429    5.111111   
8              [0.1, 0.2)            1.000000     13.750000   13.750000   
9              [0.2, 0.3)            1.000000     20.333333   20.333333   
10             [0.3, 0.4)            1.000000     16.250000   16.250000   
11             [0.4, 0.5)            1.000000     14.555556   14.555556   
12             [0.5, 0.6)

In [52]:
df.to_sql("WIN_COUNT_INCREASE_PROBS", engine, if_exists="replace", index=False)
df.to_csv("Data/win_count_increase_probs.csv")